In [10]:
%run Generation.ipynb


|.|.|.|.|.|.|.|#|.|.|.|.|.|#|.|.|.|#|.|.|#|.|.|.|.|.|.|.|#|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|#|.|#|.|.|.|.|.|.|.|.|.|
|#|#|.|#|#|#|.|#|.|#|.|#|#|#|.|.|.|#|.|.|.|.|.|.|#|.|.|.|.|#|.|#|#|#|#|#|#|#|#|.|#|#|#|.|#|.|#|#|#|.|#|.|#|#|#|#|.|#|#|#|
|.|#|.|.|#|.|.|.|.|#|.|.|.|#|.|.|.|#|.|.|#|.|.|.|#|.|.|.|#|.|.|.|.|.|#|.|.|.|.|.|.|.|#|.|#|.|.|.|#|.|.|.|#|.|#|.|.|#|.|.|
|.|.|.|.|#|.|.|#|.|#|.|.|.|#|#|.|#|#|#|#|.|#|#|#|#|.|#|#|#|.|.|.|.|.|.|.|.|.|.|.|.|.|#|.|.|.|.|.|#|.|#|.|.|.|.|.|.|#|.|.|
|.|#|#|.|#|.|.|#|.|#|.|#|#|#|.|.|.|#|.|.|.|#|.|.|#|.|.|.|#|#|#|#|#|#|#|#|#|#|#|.|#|#|#|#|.|#|#|#|#|.|#|.|#|.|#|.|.|.|.|.|
|.|#|.|.|.|.|.|#|.|#|.|.|.|#|#|.|#|#|.|.|.|.|.|.|#|.|.|.|#|.|.|.|.|.|.|.|.|#|.|#|.|.|#|.|.|.|.|.|#|.|#|#|#|#|#|#|.|#|#|#|
|.|#|.|.|#|.|.|#|.|#|.|#|#|#|.|.|.|#|.|.|.|#|.|.|#|.|.|.|#|.|.|.|#|.|#|.|.|.|.|.|.|.|#|.|.|#|.|.|#|.|#|.|.|.|.|.|.|.|.|.|
|.|.|.|.|#|.|.|#|.|.|.|.|.|#|.|.|.|.|.|#|#|#|#|.|#|.|.|.|#|.|.|.|#|.|#|.|.|#|.|#|.|.|#|.|#|#|#|.|.|.|#|#|.|#|#|#|#|#|.|#|
|.|#|.|.|#|.|.|#|.|#|.|.

# First attempt (using arcade)

In [7]:
import arcade
import numpy as np
# %run Generation.ipynb

# Set constants for the screen size
SCREEN_WIDTH = 600
SCREEN_HEIGHT = 600

# maze = [
#     ["#", ".", ".", ".", ".", ".", ".", ".", ".", "#", "."],
#     [".", ".", "#", "#", "#", "#", "#", ".", ".", ".", "."],
#     [".", "#", "#", ".", ".", ".", ".", ".", "#", ".", "."],
#     [".", ".", "#", ".", ".", ".", ".", ".", "#", ".", "."],
#     ["#", ".", "#", ".", ".", ".", ".", ".", "#", ".", "#"],
#     [".", ".", "#", ".", ".", ".", ".", ".", ".", ".", "."],
#     [".", "#", "#", "#", "#", ".", ".", "#", ".", "#", "."],
#     [".", "#", ".", ".", "F", "#", ".", "#", "#", "#", "."],
#     [".", "#", ".", "#", "#", "#", "#", "#", ".", ".", "."],
#     [".", "#", ".", "#", ".", ".", ".", ".", ".", "#", "."],
#     [".", "#", ".", "#", "#", "#", "#", "#", ".", "#", "."],
#     [".", "#", ".", ".", ".", "#", ".", ".", ".", ".", "."],
#     [".", "#", "#", "#", ".", "#", ".", "#", "#", ".", "."],
#     ["S", "#", ".", ".", ".", ".", ".", ".", ".", "#", "."]
# ]


class MyGame(arcade.Window):
    """ Main application class. """

    def __init__(self, maze, width, height):
        super().__init__(width, height)
        
        self.width = width
        self.height = height
        self.maze = maze
        self.stats = maze.stats
        arcade.set_background_color(arcade.color.AMAZON)

    def setup(self):
        # Set up your game here
        self.state = "Menu" # Menu, Daily, Levels, Success, Fail, Credits
        
        self.padding = 10
        self.charW = (self.width - self.padding * 2) / self.maze.width
        self.charH = (self.height - self.padding * 2) / self.maze.height
        
        #Start placement
        for key in self.stats:
            if self.stats[key] == "S":
                self.player = key
                print((key[0] * self.charW + self.padding, key[1] * self.charH + self.padding))
                self.playerPos = (key[0] * self.charW + self.padding, key[1] * self.charH + self.padding)
                    
        pass

    def on_draw(self):
        playerPos = self.playerPos
        """ Render the screen. """
        arcade.start_render()
        # Your drawing code goes here
        
        #Draw border
        arcade.draw_lrtb_rectangle_filled(0, self.width, self.height, 0, arcade.color.GRAY)
        # Draw maze area
        arcade.draw_lrtb_rectangle_filled(self.padding, self.width - self.padding, self.height - self.padding, self.padding, arcade.color.WHITE)
        # Draw player
        arcade.draw_lrtb_rectangle_filled(playerPos[0], playerPos[0] + self.charW, playerPos[1] + self.charH, playerPos[1], arcade.color.GREEN)
        
        # Draw walls and finish
        for key in self.stats:
            xPos = key[0] * self.charW + self.padding
            yPos = key[1] * self.charH + self.padding
            if self.stats[key] == "#":
                arcade.draw_lrtb_rectangle_filled(xPos, xPos + self.charW, yPos + self.charH, yPos, arcade.color.BLACK)
            if self.stats[key] == "F":
                arcade.draw_lrtb_rectangle_filled(xPos, xPos + self.charW, yPos + self.charH, yPos, arcade.color.RED)
                    
        

    def update(self, delta_time):
        """ the game logic goes here. """
        pass
    
    def on_key_press(self, key, modifiers):
        """Called whenever a key is pressed. """
        if key == arcade.key.UP and self.player[1] + 1 < self.maze.height and self.stats[(self.player[0], self.player[1] + 1)] != "#":
            self.player = (self.player[0], self.player[1] + 1)
            self.playerPos = (self.playerPos[0], self.playerPos[1] + self.charH)
        elif key == arcade.key.DOWN and self.player[1] - 1 >= 0 and self.stats[(self.player[0], self.player[1] - 1)] != "#":
            self.player = (self.player[0], self.player[1] - 1)
            self.playerPos = (self.playerPos[0], self.playerPos[1] - self.charH)
        elif key == arcade.key.LEFT and self.player[0] - 1 >= 0 and self.stats[(self.player[0] - 1, self.player[1])] != "#":
            self.player = (self.player[0] - 1, self.player[1])
            self.playerPos = (self.playerPos[0] - self.charW, self.playerPos[1])
        elif key == arcade.key.RIGHT and self.player[0] + 1 < self.maze.width and self.stats[(self.player[0] + 1, self.player[1])] != "#":
            self.player = (self.player[0] + 1, self.player[1])
            self.playerPos = (self.playerPos[0] + self.charW, self.playerPos[1])


def main():
    game = MyGame(g1, SCREEN_WIDTH, SCREEN_HEIGHT)
    game.setup()
    arcade.run()


if __name__ == "__main__":
    main()


(207.2, 207.2)


KeyboardInterrupt: 

# 2nd Version - Built into generation file
Pretty slow but cool looking

In [11]:
g1 = graph(maze)
w = 60
h = 36
g1.make_room_maze(w, h)
g1.optimize_maze()
print(g1.start)
g1.print_graph()
arr = dictToArr(g1.stats,w,h)
game1 = MazeLevel(g1)
game1.playLevel()

(0, 6)
|.|.|.|.|.|.|.|.|.|.|.|.|.|#|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|
|.|#|.|#|#|#|#|.|#|#|#|#|.|#|.|.|.|.|#|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|
|.|#|.|.|.|#|.|.|.|.|#|.|.|#|#|#|.|#|#|#|#|#|#|#|#|#|#|#|#|#|#|#|.|#|#|#|#|#|#|#|#|#|#|#|#|#|#|#|#|#|#|#|#|#|#|#|#|#|#|#|
|.|.|.|.|.|#|.|.|.|.|.|.|.|#|.|#|.|.|#|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|#|.|#|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|
|#|.|#|#|#|#|#|#|#|.|#|#|#|#|.|#|.|.|#|#|.|#|#|#|#|.|#|.|#|#|#|#|#|#|.|#|#|#|#|.|#|#|#|#|#|#|#|#|#|.|#|#|#|#|#|#|.|#|#|#|
|.|.|.|.|.|#|.|.|.|#|.|.|.|#|.|.|.|.|#|.|.|.|.|.|.|.|#|.|.|.|.|.|.|.|.|.|.|.|.|.|#|.|.|.|.|.|.|.|.|.|.|.|.|.|#|.|.|.|.|.|
|S|#|.|.|.|#|#|#|.|#|.|.|.|#|.|#|.|#|#|.|.|#|.|.|.|.|#|.|.|.|.|.|.|#|.|.|.|.|#|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|
|.|#|.|.|.|#|.|.|.|#|.|.|.|.|.|#|.|.|.|#|#|#|.|#|#|#|#|#|#|#|#|#|#|#|#|#|.|#|#|.|#|#|#|#|#|.|#|#|#|#|#|#|#|#|#|#|#|#|#|#|
|.|#|.|.|.|#|.|.|

Terminator: 